In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv('archive/audi.csv')
df = df.drop(['model', 'transmission', 'fuelType'], axis=1)
df.head()

,year,price,mileage,tax,mpg,engineSize
0,2017,12500,15735,150,55.4,1.4
1,2016,16500,36203,20,64.2,2.0
2,2016,11000,29946,30,55.4,1.4
3,2017,16800,25952,145,67.3,2.0
4,2019,17300,1998,145,49.6,1.0


In [3]:
x = df.drop(['price'], axis=1)
y = df['price']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [4]:
DT = DecisionTreeClassifier(max_depth=100, max_features=3, random_state=42)
DT.fit(x_train, y_train)
y_pred = DT.predict(x_test)
r2_score(y_test, y_pred)

0.8957738817911829

In [5]:
df1 = pd.read_csv('archive/audi.csv')
df1 = pd.concat([df1, pd.read_csv('archive/bmw.csv')])
df1 = pd.concat([df1, pd.read_csv('archive/cclass.csv')])
df1 = pd.concat([df1, pd.read_csv('archive/focus.csv')])
df1 = pd.concat([df1, pd.read_csv('archive/ford.csv')])
df1 = pd.concat([df1, pd.read_csv('archive/hyundi.csv').rename(columns={'tax(£)' : 'tax'})])
df1 = pd.concat([df1, pd.read_csv('archive/merc.csv')])
df1 = pd.concat([df1, pd.read_csv('archive/skoda.csv')])
df1 = pd.concat([df1, pd.read_csv('archive/toyota.csv')])
df1 = pd.concat([df1, pd.read_csv('archive/vauxhall.csv')])
df1 = pd.concat([df1, pd.read_csv('archive/vw.csv')])
df1.fillna(0, inplace=True)
df1 = df1.sample(n=10000)
df1

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
4477,Golf,2015,9500,Manual,44079,Petrol,160.0,44.1,1.4
13047,Insignia,2015,7899,Manual,49585,Diesel,0.0,76.3,2.0
8623,Kuga,2017,14290,Manual,23444,Diesel,145.0,64.2,1.5
9275,1 Series,2015,8290,Manual,35000,Petrol,125.0,51.4,1.6
2391,Focus,2018,16999,Manual,10983,Petrol,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
7456,3 Series,2016,17500,Automatic,48000,Other,0.0,134.5,2.0
5919,C-HR,2017,20690,Automatic,18851,Hybrid,0.0,74.3,1.8
8238,GLE Class,2020,61489,Semi-Auto,1000,Petrol,145.0,29.5,3.0
1029,C Class,2017,27526,Automatic,15958,Petrol,0.0,0.0,3.0


In [6]:
x = df1.drop(['price'], axis=1)
y = df1['price']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)

In [7]:
estimators = {
    'DT': DecisionTreeClassifier(),
    'RF': RandomForestClassifier()
}

params = {
    'DT': {
        'max_depth': [10, 20, 30, 40, 50],
        'max_features': [10, 100, 200],
        'min_samples_split': [2, 5, 10, 20],
        'criterion': ['gini', 'entropy']
}, 
    'RF': {
        'n_estimators': [100, 500],
        'criterion': ['gini', 'entropy'],
        'max_depth': [5, 100, 150]
}}

for name, model in estimators.items():
    m = RandomizedSearchCV(model, params[name], random_state=42)
    m.fit(x_train, y_train)
    m.predict(x_test)
    print(name, m.best_params_, m.best_score_)
    print('r2', r2_score(y_test, y_pred))

/Users/timofejsosnin/anaconda3/envs/classical-ml/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/timofejsosnin/anaconda3/envs/classical-ml/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:542: FitFailedWarning: 
10 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/timofejsosnin/anaconda3/envs/classical-ml/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Us

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- model_ A2
- model_ Fusion
- model_ M5
- model_ R8
Feature names seen at fit time, yet now missing:
- model_ 8 Series
- model_ California
- model_ Caravelle
- model_ Eos
- model_ G Class
- ...
